In [1]:
from transformers import GPT2Tokenizer
import sentencepiece as spm

In [7]:
from Bio import SeqIO
path = '/data/rozen/home/e0833634/lama/data/uniprot/'
with open(path+"uniref50.fasta", "r") as fasta_file, open("protein_corpus.txt", "w") as txt_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        txt_file.write(str(record.seq) + "\n")

In [ ]:
# Define the training arguments
spm_args = '--input=/data/rozen/home/e0833634/lama/protllama/notebooks/protein_corpus.txt --model_prefix=protein --vocab_size=32000'

# Train the model
spm.SentencePieceTrainer.Train(spm_args)

In [11]:
!export PATH="$PATH:/data/rozen/home/e0833634/.local/bin/"

In [12]:
!spm_train --input=protein_corpus.txt --model_prefix=protein --vocab_size=32000 --character_coverage=1.0 --model_type=bpe

/bin/bash: spm_train: command not found


In [2]:
tokenizer_bpe = GPT2Tokenizer.from_pretrained("/data/rozen/home/e0833634/lama/protgpt2")
tokenizer_bpe

GPT2Tokenizer(name_or_path='/data/rozen/home/e0833634/lama/protgpt2', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [3]:
import pickle
with open('/data/rozen/home/e0833634/lama/protllama/uniprot_dataset.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [4]:
train_ds = loaded_data['train']
train_ds

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 8716143
})

In [5]:
def detokenize_function(examples):
    return {"text": tokenizer_bpe.decode(examples['input_ids'], skip_special_tokens=True)}

detokenized_dataset = train_ds.map(detokenize_function)

Map:   0%|          | 0/8716143 [00:00<?, ? examples/s]

2023-09-21 20:06:46.464996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load('path_to_sentencepiece_model.model')

def tokenize_function(examples):
    return {"input_ids": sp.EncodeAsIds(examples['text'])}

dataset_tokenized_sp = detokenized_dataset.map(tokenize_function)
